In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle 
import pandas as pd
import numpy as np

In [44]:
# load model and scaler


model = load_model('model.h5')
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

with open('one_hote_encoder_pickle.pkl', 'rb') as file:
    geography_encoder = pickle.load(file)
    

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

In [45]:
#Example input data 

input_data = {
    'CreditScore': 608,
    'Geography': 'Spain',
    'Gender': 'Female',
    'Age': 41,
    'Tenure': 1,
    'Balance': 83807.86,
    'NumOfProducts': 1,
    'HasCrCard': 0,
    'IsActiveMember': 1,
    'EstimatedSalary': 112542.58
}

In [46]:
geo_encoded = geography_encoder.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=geography_encoder.get_feature_names_out(['Geography']))
geo_encoded_df

/home/sikander/miniconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,0.0,0.0,1.0


In [47]:
input_df = pd.DataFrame(input_data, index=[0])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,608,Spain,Female,41,1,83807.86,1,0,1,112542.58


In [48]:
input_data_encoded = pd.concat([input_df.reset_index(drop=True), geo_encoded_df], axis=1)
input_data_encoded

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0


In [49]:
input_data_encoded['Gender'] = label_encoder_gender.transform([input_data_encoded['Gender']])
input_data_encoded

/home/sikander/miniconda3/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0


In [50]:
input_data_encoded = pd.DataFrame(input_data_encoded)
input_data_encoded

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0


In [51]:
input_data_encoded = input_data_encoded.drop('Geography', axis=1)
input_data_encoded

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,608,0,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0


In [52]:
input_data_encoded = scaler.transform(input_data_encoded)
input_data_encoded

array([[-0.45296331, -1.09499335,  0.19986603, -1.38944225,  0.12337621,
        -0.91668767, -1.54035103,  0.97481699,  0.21057501, -0.99850112,
        -0.57946723,  1.73494238]])

In [53]:
scaled_data = input_data_encoded
scaled_data

array([[-0.45296331, -1.09499335,  0.19986603, -1.38944225,  0.12337621,
        -0.91668767, -1.54035103,  0.97481699,  0.21057501, -0.99850112,
        -0.57946723,  1.73494238]])

In [54]:
prediction = model.predict(scaled_data)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step


array([[0.18937428]], dtype=float32)

In [55]:
prediciton_prob = prediction[0][0]


In [56]:
prediciton_prob

0.18937428

In [57]:
if prediciton_prob > 0.5: 
    print('The customer is likely to leave the bank')
else: 
    print('Customer is not likely to leave the bank')

Customer is not likely to leave the bank
